In [42]:
import numpy as np
import itertools

In [43]:
V = 6
E = 5
colors = "3"
properedges  = np.load("./data/properedges_"+colors+"color.npy")

In [44]:
def index(i,j,k):
    idx_list = list(itertools.combinations(np.arange(V), 3))
    tup = tuple(sorted((i,j,k)))
    return idx_list.index(tup)

In [45]:
idx_list = list(itertools.combinations(np.arange(V), 3))
N = len(idx_list)
print(N) # maximum number of edges

20


In [46]:
def kbits(n, k):
    result = []
    for bits in itertools.combinations(range(n), k):
        s = ['0'] * n
        for bit in bits:
            s[bit] = '1'
        result.append(''.join(s))
    return result

In [47]:
all_graphsb = [kbits(N, i) for i in range(E)]
all_graphsb = [i for s in all_graphsb for i in s]

## Generate all Isoclasses

In [48]:
def edge():
    for i in range(20):
        yield '0'*i + '1' + '0'*(19-i)

In [49]:
def twoPath2():
    for i,j in itertools.combinations(range(V),2):
        for l,m in itertools.combinations(set(range(V))-{i,j},2):
            a = index(i,j,l)
            b = index(i,j,m)
            if a>b: a, b = b, a
            yield '0'*a + '1' + '0'*(b-a-1) + '1' + '0'*(19-b)

In [50]:
def twoEdges():
    for i,j,k in itertools.combinations(range(V),3):
        for l,m,n in itertools.combinations(set(range(V))-{i,j,k},3):
            a = index(i,j,k)
            b = index(l,m,n)
            if a>b: a, b = b, a
            yield '0'*a + '1' + '0'*(b-a-1) + '1' + '0'*(19-b)
    for i in range(V):
        for j,k,m,n in itertools.combinations(set(range(V))-{i},4):
            for j,k,m,n in [(j,k,m,n),(j,m,k,n),(j,n,k,m)]:
                a = index(i,j,k)
                b = index(i,m,n)
                if a>b: a, b = b, a
                yield '0'*a + '1' + '0'*(b-a-1) + '1' + '0'*(19-b)

In [51]:
def threeS122():
    for i in range(V):
        for j,k in itertools.combinations(set(range(V))-{i},2):
            for m in set(range(V))-{i,j,k}:
                for n in set(range(V))-{i,j,k,m}:
                    a = index(i,j,k)
                    b = index(i,j,m)
                    c = index(i,k,n)
                    a,b,c = sorted([a,b,c])
                    yield '0'*a + '1' + '0'*(b-a-1) + '1' + '0'*(c-b-1) + '1' + '0'*(19-c)


In [52]:
def fourS1222():
    for i in range(V):
        for j,k in itertools.combinations(set(range(V))-{i},2):
            for m in set(range(V))-{i,j,k}:
                for n in set(range(V))-{i,j,k,m}:
                    a = index(i,j,k)
                    b = index(i,j,m)
                    c = index(i,k,n)
                    d = index(i,m,n)
                    a,b,c,d = sorted([a,b,c,d])
                    yield '0'*a + '1' + '0'*(b-a-1) + '1' + '0'*(c-b-1) + '1' + '0'*(d-c-1) + '1' + '0'*(19-d)

In [53]:
def threeCycle2():
    for i in range(V):
        for j,k,m in itertools.combinations(set(range(V))-{i},3):
            a = index(i,j,k)
            b = index(i,j,m)
            c = index(i,k,m)
            a,b,c = sorted([a,b,c])
            yield '0'*a + '1' + '0'*(b-a-1) + '1' + '0'*(c-b-1) + '1' + '0'*(19-c)

In [54]:
def fourCycle1212():
    for i,j in itertools.combinations(range(V),2):
        for k,l in itertools.combinations(set(range(V))-{i,j},2):
            for m in set(range(V))-{i,j,k,l}:
                for n in set(range(V))-{i,j,k,l,m}:
                    a = index(i,j,n)
                    b = index(i,j,m)
                    c = index(k,l,n)
                    d = index(k,l,m)
                    a,b,c,d = sorted([a,b,c,d])
                    yield '0'*a + '1' + '0'*(b-a-1) + '1' + '0'*(c-b-1) + '1' + '0'*(d-c-1) + '1' + '0'*(19-d)

In [55]:
def threeCycle1inscribed():
    for i,j,k in itertools.combinations(range(V),3):
        for m in set(range(V))-{i,j,k}:
            for n in set(range(V))-{i,j,k,m}:
                for o in set(range(V))-{i,j,k,m,n}:
                    a = index(i,j,m)
                    b = index(j,k,n)
                    c = index(k,i,o)
                    d = index(m,n,o)
                    a,b,c,d = sorted([a,b,c,d])
                    yield '0'*a + '1' + '0'*(b-a-1) + '1' + '0'*(c-b-1) + '1' + '0'*(d-c-1) + '1' + '0'*(19-d)

In [62]:
ISOCLASSES = ["0"*20,
              list(edge()),
              list(twoPath2()),
              list(twoEdges()),
              list(threeCycle2()),
              list(threeS122()),
              list(fourS1222()),
              list(fourCycle1212()),
              list(threeCycle1inscribed())
              ]

## Generate the coefficient matrix

In [63]:
all_subgraphs = np.array([['{0:020b}'.format(int(edges,2) & int(mono,2)) for mono in properedges] for edges in all_graphsb])

In [64]:
mat = np.zeros((all_subgraphs.shape[0], len(ISOCLASSES)))

for i in range(len(all_subgraphs)):
    for j in range(len(ISOCLASSES)): # contains the edge list of all possible graphs in the isoclass
        if ISOCLASSES[j] is None:
            mat[i][j]=0
        else:
            mat[i][j] = np.count_nonzero([subgraph in ISOCLASSES[j] for subgraph in all_subgraphs[i]])

## Compute Mu

In [65]:
import cvxpy as cp
import numpy as np

In [66]:
# maximize delta (away from 0)
x = cp.Variable(len(ISOCLASSES)+1)
prob = cp.Problem(cp.Maximize(x[0]),
                 [ cp.abs((mat[1:,:]/243) @ x[1:]) <= np.ones(mat.shape[0]-1)*(1-x[0]),
                 x[[1,2,3,5]] == np.array([15,-10.2,0.6,-6.6]),
                 x[0]>=0,
                 ])
result = prob.solve()
print(result)
print(x.value)

1.4941105326662704e-15
[ 1.49411053e-15  1.50000000e+01 -1.02000000e+01  6.00000000e-01
  3.82186204e+00 -6.60000000e+00  5.16120948e-01 -1.15735799e+01
 -9.97498141e+00 -1.18544732e+01]


In [67]:
# minimize sum of absolute of c_H
x = cp.Variable(len(ISOCLASSES))
prob = cp.Problem(cp.Minimize(cp.sum(cp.abs(x))),
                 [ cp.abs((mat[1:,:]/243) @ x) <= np.ones(mat.shape[0]-1),
                 x[0] == 15
                #  x[[0,1,2,4]] == np.array([15,-10.2,0.6,-6.6])
                 ])
result = prob.solve()
print(result)
print(x.value)

42.95382377616706
[ 1.50000000e+01 -1.02000000e+01  6.00000000e-01  3.48000000e+00
 -6.60000000e+00 -6.21741819e-13 -4.55076923e+00 -1.86545455e+00
 -6.57600000e-01]


## Test Coefficients

In [68]:
coeff = [15,-10.2,0.6,3.48,-6.6,0,-10,-10,-10]

In [69]:
np.set_printoptions(suppress=True)
mu = np.abs(mat/243 @ np.array(coeff).reshape(-1,1)).flatten()
print((mu[1:]<=1).all())
mu

False


array([15.        ,  1.        ,  1.        , ...,  0.00888889,
        0.00888889,  1.        ])

In [70]:
# print graphs that have mu > 1
(mu>1+1e-9).nonzero()

(array([0]),)

In [71]:
for i,a in enumerate(all_graphsb[2088]):
    if a == '1':
        print(idx_list[i])

(0, 1, 2)
(0, 3, 4)
(1, 3, 5)
(2, 4, 5)


In [72]:
idx_list

[(0, 1, 2),
 (0, 1, 3),
 (0, 1, 4),
 (0, 1, 5),
 (0, 2, 3),
 (0, 2, 4),
 (0, 2, 5),
 (0, 3, 4),
 (0, 3, 5),
 (0, 4, 5),
 (1, 2, 3),
 (1, 2, 4),
 (1, 2, 5),
 (1, 3, 4),
 (1, 3, 5),
 (1, 4, 5),
 (2, 3, 4),
 (2, 3, 5),
 (2, 4, 5),
 (3, 4, 5)]